In [ ]:
Build recommender System using Dataset acquired from GroupLens.

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
movies_df = pd.read_csv('E:/abhi/Desktop/Intel-HPC/ml_algos/Databases/movies.csv')
ratings_df = pd.read_csv('E:/abhi/Desktop/Intel-HPC/ml_algos/Databases/ratings.csv')
movies_df.head()

In [ ]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

In [ ]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

In [ ]:
moviesWithGenres_df = movies_df.copy()

for index, row in movies_df.iterrows():  
    for genre in row['genres']:  
        moviesWithGenres_df.at[index, genre] = 1
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

In [ ]:
## Rating frame
ratings_df.head()

In [ ]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

## Content-Based recommendation system

In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

In [ ]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]  
inputMovies = pd.merge(inputId, inputMovies)  
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)  
inputMovies

In [ ]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

In [ ]:
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

In [ ]:
inputMovies['rating']

In [ ]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])

userProfile

In [ ]:

genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])  
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable

In [ ]:
genreTable.shape 

In [ ]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())

recommendationTable_df.head()

## To understand previous cell, sample code

In [ ]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()
 

## Recommendation Table

In [ ]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]